Portfolio 6

In [1]:
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(f"Using device: {device}")

Using device: cpu


Step 1

In [2]:
import pandas as pd 
import os 
from ultralytics import YOLO 
from tqdm import tqdm 
 

def convert_annotations(csv_file, images_dir, output_dir, class_mapping): 
    df = pd.read_csv(csv_file) 
    grouped = df.groupby('filename')
    
    if not os.path.exists(output_dir): 
        os.makedirs(output_dir) 
    
    for filename, group in tqdm(grouped, desc=f'Converting annotation for {csv_file}'): 
        image_path = os.path.join(images_dir, filename) 
        if not os.path.exists(image_path): 
            continue 
        
        img_width = group.iloc[0]['width'] 
        img_height = group.iloc[0]['height'] 
        
        annotation = [] 
        for _, row in group.iterrows(): 
            class_id = class_mapping[row['class']] 
            xmin = row['xmin'] 
            ymin = row['ymin'] 
            xmax = row['xmax'] 
            ymax = row['ymax'] 
            
            x_center = ((xmin + xmax) / 2) / img_width 
            y_center = ((ymin + ymax) / 2) / img_height 
            bbox_width = (xmax - xmin) / img_width 
            bbox_height = (ymax - ymin) / img_height 
            
            annotation.append(f"{class_id}, {x_center} {y_center} {bbox_width} {bbox_height}") 
            
        txt_filename = os.path.splitext(filename)[0] + '.txt' 
        with open(os.path.join(output_dir, txt_filename), 'w') as f: 
            for ann in annotation: 
                f.write(ann + '\n')

In [3]:
class_mapping = {'Graffiti': 0} 

convert_annotations(r'C:\Users\cucum\Downloads\COS40007/week 6/Bounding_boxes/train_labels.csv', 
                    r'C:\Users\cucum\Downloads\COS40007/week 6/images/train',  
                    r'C:\Users\cucum\Downloads\COS40007/week 6/labels/train',  
                    class_mapping) 

convert_annotations(r'C:\Users\cucum\Downloads\COS40007/week 6/Bounding_boxes/test_labels.csv',  
                    r'C:\Users\cucum\Downloads\COS40007/week 6/images/test',  
                    r'C:\Users\cucum\Downloads\COS40007/week 6/labels/test',  
                    class_mapping)

Converting annotation for C:\Users\cucum\Downloads\COS40007/week 6/Bounding_boxes/train_labels.csv: 100%|██████████| 813/813 [00:00<00:00, 1374.43it/s]
Converting annotation for C:\Users\cucum\Downloads\COS40007/week 6/Bounding_boxes/test_labels.csv: 100%|██████████| 209/209 [00:00<00:00, 1229.93it/s]


In [4]:
import random 
import shutil
def select_random_images(source_dir, dest_dir, num_images, used_images=set()): 
    images = [f for f in os.listdir(source_dir) if f.endswith('.jpg') and f not in used_images]  
    selected_images = random.sample(images, min(num_images, len(images))) 
    used_images.update(selected_images) 
    
    if not os.path.exists(dest_dir): 
        os.makedirs(dest_dir) 
    
    for img in selected_images: 
        shutil.copy(os.path.join(source_dir, img), os.path.join(dest_dir, img)) 
    return used_images 

def copy_annotation_files(image_dir, label_source_dir, label_dest_dir): 
    if not os.path.exists(label_dest_dir): 
        os.makedirs(label_dest_dir) 
    for img_file in os.listdir(image_dir): 
        if img_file.endswith('.jpg'): 
            base_name = os.path.splitext(img_file)[0] 
            label_file = base_name + '.txt' 
            src_label_path = os.path.join(label_source_dir, label_file) 
            dst_label_path = os.path.join(label_source_dir, label_file)
            if os.path.exists(src_label_path): 
                shutil.copy(src_label_path, dst_label_path)

Step 2

In [5]:
random.seed(42) 

used_train_images = set() 
 
used_train_images = select_random_images(r'C:\Users\cucum\Downloads\COS40007/week 6/images/train',  
                                         r'C:\Users\cucum\Downloads\COS40007/images/train_selected', 400, used_train_images) 

copy_annotation_files(r'C:\Users\cucum\Downloads\COS40007/week 6/images/train', 
                      r'C:\Users\cucum\Downloads\COS40007/week 6/label/train',  
                      r'C:\Users\cucum\Downloads\COS40007/week 6/images/train_selected')

Step 3

In [6]:
random.seed(42)  

used_test_images = set() 

used_test_images = select_random_images(r'C:\Users\cucum\Downloads\COS40007/week 6/images/test',  
                                         r'C:\Users\cucum\Downloads\COS40007/week 6/images/test_selected', 40, used_test_images)  

copy_annotation_files(r'C:\Users\cucum\Downloads\COS40007/week 6/images/test', 
                      r'C:\Users\cucum\Downloads\COS40007/week 6/label/test',  
                      r'C:\Users\cucum\Downloads\COS40007/week 6/images/test_selected')

In [7]:
from torchvision.ops import box_iou   
import cv2
def compute_iou(pred_boxes, true_boxes):  
    pred_boxes = torch.tensor(pred_boxes) 
    true_boxes = torch.tensor(true_boxes) 
    
    iou = box_iou(pred_boxes, true_boxes) 
    return iou.diag().numpy() 

def evaluate_model(model, images_dir, labels_dir, output_images_dir=None): 
    results = [] 
    images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')] 
    
    if output_images_dir and not os.path.exists(output_images_dir): 
        os.makedirs(output_images_dir) 
        
    for img_name in tqdm(images, desc='Evaluating model'): 
        img_path = os.path.join(images_dir, img_name) 
        label_path = os.path.join(labels_dir, os.path.splitext(img_name)[0] + '.txt') 
        
        preds = model.predict(img_path, conf=0.25) 
        
        img = cv2.imread(img_path)  
        img_height, img_width = img.shape[:2] 
        
        pred_boxes = [] 
        confidence = [] 
        for pred in preds: 
            for box in pred_boxes: 
                x_min = box.xyxy[0][0].item() 
                x_max = box.xyxy[0][1].item() 
                y_min = box.xyxy[0][2].item() 
                y_max = box.xyxy[0][3].item() 
                conf = box.conf.item() 
                pred_boxes.append([x_min, y_min, x_min, x_max]) 
                confidence.append(conf) 
                if output_images_dir: 
                    label = f"{model.names[int(box.cls)]}: {conf:.2f}" 
                    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2) 
                    cv2.putText(img, label,(int(x_min), int(y_min) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  
                    
        true_boxes = [] 
        if os.path.exists(label_path): 
            with open(label_path, 'r') as f: 
                for line in f: 
                    class_id, x_center, y_center, width, height = map(float, line.strip().split()) 
                    x_center *= img_width 
                    y_center *= img_height 
                    width *= img_width 
                    height *= img_height 
                    x_min = x_center -  width / 2 
                    y_min = y_center - height / 2 
                    x_max = x_center + width / 2 
                    y_max = y_center + height / 2 
                    true_boxes.append([x_min, y_min, x_max, y_max]) 
                    
                    if output_images_dir: 
                        cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 0, 255), 2) 
                        
            if output_images_dir: 
                output_images_dir = os.path.join(output_images_dir, img_name) 
                cv2.imwrite(output_images_dir, img) 
            
            if pred_boxes and true_boxes: 
                ious = compute_iou(pred_boxes, true_boxes) 
                max_iou = max(ious) 
                max_conf = confidence[ious.argmax()] 
            else: 
                max_iou = 0.0 
                max_conf = 0.0 if not confidence else max(confidence) 
                
            results.append({
            'image_name': img_name,
            'confidence_value': max_conf,
            'IoU_value': max_iou
        }) 
            
    return pd.DataFrame(results)

In [8]:
def create_yaml_file(file_path, train_images, val_images, nc, class_names): 
    data = {  
            'train': train_images, 
            'val' : val_images, 
            'nc' : nc, 
            'names' : class_names
            } 
    with open(file_path, 'w') as f: 
        for key, value in data.items(): 
            if isinstance(value, list): 
                value_str = str(value).replace("'", "") 
                f.write(f"{key}: {value_str}\n") 
            else: 
                f.write(f"{key}: {value}\n")

In [9]:
yaml_file_path = 'C:\\Users\\cucum\\yolov5\\graffiti.yaml' 
train_images_path = os.path.abspath(r'C:\Users\cucum\Downloads\COS40007/week 6/images/train_selected') 
val_images_path = os.path.abspath(r'C:\Users\cucum\Downloads\COS40007/week 6/images/test_selected') 
nc = 1 
class_name = ['Graffiti'] 
create_yaml_file(yaml_file_path, train_images_path, val_images_path, nc, class_name)

In [10]:
from ultralytics import YOLO 
model = YOLO('yolov5su.pt') 
results = model.train(data=yaml_file_path, epochs=4, imgsz=640, batch=16, name='graffiti_detection', device=device)

100%|██████████| 17.7M/17.7M [00:00<00:00, 70.3MB/s]


engine\trainer: task=detect, mode=train, model=yolov5su.pt, data=C:\Users\cucum\yolov5\graffiti.yaml, epochs=4, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=graffiti_detection6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, opt

train: Scanning C:\Users\cucum\Downloads\COS40007\week 6\labels\train_selected... 0 images, 400 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<00:00, 2767.87it/s]

train: WARNING  No labels found in C:\Users\cucum\Downloads\COS40007\week 6\labels\train_selected.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


train: New cache created: C:\Users\cucum\Downloads\COS40007\week 6\labels\train_selected.cache
WARNING  No labels found in C:\Users\cucum\Downloads\COS40007\week 6\labels\train_selected.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...

                   all         71          0          0          0          0          0
WARNING  no labels found in detect set, can not compute metrics without labels



4 epochs completed in 0.224 hours.
Optimizer stripped from runs\detect\graffiti_detection6\weights\last.pt, 18.5MB
Optimizer stripped from runs\detect\graffiti_detection6\weights\best.pt, 18.5MB

Validating runs\detect\graffiti_detection6\weights\best.pt...
Ultralytics YOLOv8.2.98  Python-3.12.6 torch-2.4.1+cpu CPU (12th Gen Intel Core(TM) i7-1260P)
YOLOv5s summary (fused): 193 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:07<00:00,  2.44s/it]

                   all         71          0          0          0          0          0
WARNING  no labels found in detect set, can not compute metrics without labels


Speed: 0.7ms preprocess, 93.9ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\detect\graffiti_detection6


In [11]:
df_results = evaluate_model(model,  
                            r'C:\Users\cucum\Downloads\COS40007/week 6/images/test_selected',  
                            r'C:\Users\cucum\Downloads\COS40007/week 6/labels/test_selected',  
                            'week-06-portfolio/evaluation_images') 
df_results.to_csv('GrafitiDetectionResults.csv', index=False)

Evaluating model:   0%|          | 0/71 [00:00<?, ?it/s]


image 1/1 C:\Users\cucum\Downloads\COS40007\week 6\images\test_selected\IMG_1234.jpg: 640x480 (no detections), 118.1ms
Speed: 2.0ms preprocess, 118.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:   1%|▏         | 1/71 [00:00<00:15,  4.53it/s]


image 1/1 C:\Users\cucum\Downloads\COS40007\week 6\images\test_selected\IMG_1240.jpg: 640x480 (no detections), 100.0ms
Speed: 2.0ms preprocess, 100.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:   3%|▎         | 2/71 [00:00<00:11,  6.26it/s]

...

Evaluating model:  97%|█████████▋| 69/71 [00:08<00:00,  7.85it/s]


image 1/1 C:\Users\cucum\Downloads\COS40007\week 6\images\test_selected\IMG_20180811_083610.jpg: 640x480 (no detections), 107.0ms
Speed: 2.0ms preprocess, 107.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:  99%|█████████▊| 70/71 [00:08<00:00,  7.91it/s]


image 1/1 C:\Users\cucum\Downloads\COS40007\week 6\images\test_selected\IMG_20180811_083858.jpg: 640x480 (no detections), 109.1ms
Speed: 2.0ms preprocess, 109.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model: 100%|██████████| 71/71 [00:08<00:00,  7.98it/s]


Step 4

In [19]:
from torchvision.ops import box_iou 
iou_threshold = 0.9 
satisfied = False 
iteration = 1 

while not satisfied: 
    print(f"\nStarting iteration: {iteration}") 
    
    training_images = random.sample(os.listdir(r'C:\Users\cucum\Downloads\COS40007/week 6/images/train'), 400) 
    selected_train_image_dir = f"images/train_selected_iter_{iteration}" 
    selected_train_labels_dir = f"labels/train_selected_iter_{iteration}"  
    if not os.path.exists(selected_train_image_dir): 
        os.makedirs(selected_train_image_dir) 
    if not os.path.exists(selected_train_labels_dir): 
        os.makedirs(selected_train_labels_dir) 
    
    for img in training_images: 
        shutil.copy(os.path.join(r'C:\Users\cucum\Downloads\COS40007/week 6/images/train', img), os.path.join(selected_train_image_dir, img)) 
        label_file = os.path.splitext(img)[0] + '.txt' 
        src_label_path = os.path.join(r'C:\Users\cucum\Downloads\COS40007/week 6/labels/train', label_file) 
        dst_label_path = os.path.join(selected_train_labels_dir) 
        if os.path.exists(src_label_path): 
            shutil.copy(src_label_path, dst_label_path) 
    
    train_images_path = os.path.abspath(selected_train_image_dir) 
    val_images_path = os.path.abspath(r'C:\Users\cucum\Downloads\COS40007/week 6/images/test_selected') 
    
    yaml_file_path_iter = f"yaml/graffiti_iter_{iteration}.yaml" 
    create_yaml_file(yaml_file_path_iter, train_images_path, val_images_path, nc, class_name) 
    
    if iteration == 1: 
        model = YOLO('yolov5su.pt') 
    else: 
        previous_model_path = '/week 6runs/train/graffiti_detection_iter_{iteration - 1}/weights/best.pt'  
        model = YOLO(previous_model_path) 
    
    model.train( 
            data=yaml_file_path_iter, 
            epochs=5, 
            imgsz=640,  
            batch=16, 
            project='week 6/runs/train',
            name=f'graffiti_detection_iter_{iteration}', 
            device=device 
    ) 
    
    # Evaluate the model
    output_images_dir_iter = f'week-06-portfolio/evaluation_images_iter_{iteration}'
    if not os.path.exists(output_images_dir_iter):
        os.makedirs(output_images_dir_iter)

    df_results = evaluate_model(model, r'C:\Users\cucum\Downloads\COS40007/week 6/images/test_selected', r'C:\Users\cucum\Downloads\COS40007/week 6/labels/test_selected', output_images_dir_iter)
    df_results.to_csv(f'week-06-portfolio/evaluation_results_iter_{iteration}.csv', index=False)

    # Check IoU threshold
    over_threshold = df_results[df_results['IoU_value'] > iou_threshold]
    if len(over_threshold) / len(df_results) >= 0.8:
        print(f"IoU threshold met in iteration {iteration}")
        satisfied = True
    else:
        print(f"IoU threshold not met in iteration {iteration}")

    # Prepare for next iteration
    iteration += 1


if not satisfied:
    print("IoU threshold not met after all iterations.")
else:
    print("Training completed successfully.")


Starting iteration: 1
engine\trainer: task=detect, mode=train, model=yolov5su.pt, data=yaml/graffiti_iter_1.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=week 6/runs/train, name=graffiti_detection_iter_16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, for

train: Scanning C:\Users\cucum\Downloads\COS40007\week 6\labels\train_selected_iter_1... 813 images, 0 backgrounds, 783 corrupt: 100%|██████████| 813/813 [00:00<00:00, 2377.97it/s]

train: WARNING  C:\Users\cucum\Downloads\COS40007\week 6\images\train_selected_iter_1\IMG_0020.JPG: ignoring corrupt image/label: could not convert string to float: '0,'
train: WARNING  C:\Users\cucum\Downloads\COS40007\week 6\images\train_selected_iter_1\IMG_0026.JPG: ignoring corrupt image/label: could not convert string to float: '0,'
train: WARNING  C:\Users\cucum\Downloads\COS40007\week 6\images\train_selected_iter_1\IMG_0027.JPG: ignoring corrupt image/label: could not convert string to float: '0,'
train: WARNING  C:\Users\cucum\Downloads\COS40007\week 6\images\train_selected_iter_1\IMG_0028.JPG: ignoring corrupt image/label: could not convert string to float: '0,'
train: WARNING  C:\Users\cucum\Downloads\COS40007\week 6\images\train_selected_iter_1\IMG_0031.JPG: ignoring corrupt image/label: could not convert string to float: '0,'
train: WARNING  C:\Users\cucum\Downloads\COS40007\week 6\images\train_selected_iter_1\IMG_0034.JPG: ignoring corrupt image/label: could not convert st

train: New cache created: C:\Users\cucum\Downloads\COS40007\week 6\labels\train_selected_iter_1.cache


val: Scanning C:\Users\cucum\Downloads\COS40007\week 6\labels\test_selected.cache... 0 images, 71 backgrounds, 0 corrupt: 100%|██████████| 71/71 [00:00<?, ?it/s]

...


image 1/1 C:\Users\cucum\Downloads\COS40007\week 6\images\test_selected\IMG_20180811_083610.jpg: 640x480 1 Graffiti, 117.9ms
Speed: 3.0ms preprocess, 117.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:  99%|█████████▊| 70/71 [00:10<00:00,  6.67it/s]


image 1/1 C:\Users\cucum\Downloads\COS40007\week 6\images\test_selected\IMG_20180811_083858.jpg: 640x480 3 Graffitis, 120.0ms
Speed: 2.0ms preprocess, 120.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model: 100%|██████████| 71/71 [00:10<00:00,  6.89it/s]


KeyError: 'IoU_value'

Step 5

In [61]:
import json
import subprocess
import re
import requests
from ultralytics import YOLO

# Initialize the YOLOv5 model
model = YOLO('yolov5su.pt')

# Your Pexels API Key
PEXELS_API = 'tRQkuTMLiGRfcBnpp9eBUY2INznat4KQbi8SbPIOv2QoQfBKGKH5LnmI'


In [50]:
# Function to extract video ID from URL
def extract_video_id(url):
    # Regex pattern to extract the video ID (the digits at the end of the URL)
    match = re.search(r'(\d+)/$', url)
    if match:
        return match.group(1)
    return None


In [51]:
# Function to get the HD video link from Pexels API
def get_hd_video_link(video_id):
    url = f"https://api.pexels.com/videos/videos/{video_id}"
    command = f'curl -H "Authorization: {PEXELS_API}" {url}'
    
    # Run the API request using subprocess
    response = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    try:
        data = json.loads(response.stdout)
        # Look for the HD link in video_files
        for video_file in data['video_files']:
            if video_file['quality'] == 'hd':
                return video_file['link']
    except json.JSONDecodeError:
        print(f"Failed to retrieve video data for ID: {video_id}")
        return None


In [52]:
# Function to download the video
def download_video(hd_link, save_path):
    try:
        response = requests.get(hd_link, stream=True)
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloaded video to: {save_path}")
        return True
    except Exception as e:
        print(f"Failed to download video: {e}")
        return False


In [53]:
# List of video URLs
urls = [
    'https://www.pexels.com/video/a-door-with-graffiti-on-it-is-shown-4543511/', 
    'https://www.pexels.com/video/busy-street-footage-854181/', 
    'https://www.pexels.com/video/graffiti-painted-on-the-train-station-wall-3413463/', 
    'https://www.pexels.com/video/a-man-doing-backflip-6962437/'
]


In [56]:
# Iterate over the URLs, extract the video ID, fetch the HD link, and track objects
for url in urls:
    video_id = extract_video_id(url)
    if video_id:
        hd_link = get_hd_video_link(video_id)
        if hd_link:
            # Download the video
            video_name = f'{video_id}.mp4'
            if download_video(hd_link, video_name):
                # Track objects in the video
                print(f"Tracking objects in {video_name}...")
                results = model.track(source=video_name, save=True, project=f'results', tracker="bytetrack.yaml")
        else:
            print(f"Failed to retrieve HD link for video ID: {video_id}")
    else:
        print(f"Could not extract video ID from URL: {url}")


Failed to retrieve video data for ID: 4543511
Failed to retrieve HD link for video ID: 4543511
Failed to retrieve video data for ID: 854181
Failed to retrieve HD link for video ID: 854181
Failed to retrieve video data for ID: 3413463
Failed to retrieve HD link for video ID: 3413463
Failed to retrieve video data for ID: 6962437
Failed to retrieve HD link for video ID: 6962437


In [60]:
# Test Run
source = '6962437-hd_1920_1080_25fps.mp4'
results = model.track(source, save=True, project=f'results', tracker="bytetrack.yaml")



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/189) c:\Users\cucum\Downloads\COS40007\week 6\6962437-hd_1920_1080_25fps.mp4: 384x640 1 person, 1 tv, 128.4ms
video 1/1 (frame 2/189) c:\Users\cucum\Downloads\COS40007\week 6\6962437-hd_1920_1080_25fps.mp4: 384x640 1 person, 1 tv, 112.8ms
video 1/1 (frame 3/189) c:\Users\cucum\Downloads\COS40007\week 6\6962437-hd_1920_1080_25fps.mp4: 384x640 1 person, 1 tv, 156.1ms
video 1/1 (frame 4/189) c:\Users\cucum\Downloads\COS40007\week 6\696243